In [38]:
options(warn = -1)

library(haven)
library(tidyverse)
library(OasisR)

options(warn = 0)

In [15]:
df = read_dta('tomp_poverty.dta')
head(df)

bgroup10,urban,fams,fpov,fnopov,fpovkids,fnopovkids
361090001001,1,136,0,136,0,45
361090002001,1,15,0,15,0,0
361090002002,1,19,0,19,0,0
361090002003,1,18,0,18,0,5
361090003001,1,33,2,31,0,0
361090004001,1,103,10,93,5,60


### 1.A)

In [25]:
df %>% 
    group_by(urban) %>%
    summarize('rate_fams' = sum(fpovkids)/sum(fams))

urban,rate_fams
0,0.07894927
1,0.06230234


In Tompkins County, poverty rates among children with families seem higher in urban areas. Given that almost all of the county population is located in the urban centre of Ithaca (I think), the absolute numbers probably show that the majority of all poverty occurs in urban areas. Let's check:

In [29]:
df %>% 
    group_by(urban) %>%
    summarize('rate_fams' = sum(fpovkids))

urban,rate_fams
0,1094
1,394


If this is coded like I think it is, then the results sure tell a different story. Looks like TC is mostly a rural area, and the urban-rural divide may have little to do with poverty, at least among those with families. Let's quickly combine and check all families:

In [32]:
df %>%
    group_by(urban) %>%
    summarize('rate_all'= sum(fpov)/sum(fams))

urban,rate_all
0,0.09222775
1,0.09203036


Almost no difference when we look across families with and without children

### 1.b)

In [56]:
m <- df %>% select(fpovkids, fnopovkids)
Eta2 <- Eta2(m)
Eta2[1]

Warning message in segdataclean(x):
"following lines were deleted because of null population: 2, 3, 5, 26, 28"

[1] 0.1953

This number value of eta-squared seems relatively low. The poor and non-poor live together in harmony.